In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

In [3]:
df = pd.read_csv("loan_data.csv")
for x in df.columns:
    if "." in (x):
        df.rename(columns={x:x.replace(".","_")} , inplace = True)
df

,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,all_other,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,all_other,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,debt_consolidation,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,home_improvement,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


In [4]:
categorical_columns = ['purpose']
numerical_columns = ['int_rate', 'installment', 'log_annual_inc',
       'dti', 'fico', 'days_with_cr_line', 'revol_bal', 'revol_util',
       'inq_last_6mths', 'delinq_2yrs', 'pub_rec']
label = "not_fully_paid"

In [5]:
df1 = df[df["not_fully_paid"] == 1].sample(1533)
df2 = df[df["not_fully_paid"] == 0].sample(1533)
dff = pd.concat([df1,df2] ,ignore_index = True)
df1 = df[df["not_fully_paid"] == 0].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([dff,df1,df2] ,ignore_index = True)
df1 = df[df["not_fully_paid"] == 0].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([dff,df1,df2] ,ignore_index = True)
df1 = df[df["not_fully_paid"] == 0].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([dff,df1,df2] ,ignore_index = True)
dff

,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,all_other,0.1071,782.44,11.156251,0.57,727,5040.041667,782,3.0,1,0,0,1
1,0,small_business,0.1442,464.17,11.695247,23.39,682,6719.958333,107404,54.5,4,0,0,1
2,0,all_other,0.1158,313.63,11.002100,7.14,732,3809.958333,5928,28.6,5,0,0,1
3,1,home_improvement,0.1407,764.64,11.813030,3.66,672,5790.000000,10753,52.3,2,3,0,1
4,1,debt_consolidation,0.1821,181.30,10.714418,11.39,662,5400.000000,0,25.8,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12259,1,debt_consolidation,0.1442,275.07,10.858999,6.88,667,3960.000000,9760,66.8,0,0,0,1
12260,1,credit_card,0.1913,918.02,11.904968,18.54,662,4890.000000,46031,77.4,1,1,0,1
12261,1,small_business,0.1316,378.23,11.063508,4.25,732,5219.958333,8897,19.1,3,0,0,1
12262,1,credit_card,0.1114,147.62,10.373491,12.98,727,2069.958333,10491,27.0,3,0,0,1


In [7]:
X = df[numerical_columns]
y = df[[label]]

In [8]:
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size = 0.33,random_state = 42, stratify = y[label])

In [9]:
model = XGBClassifier()

In [10]:
model.fit(train_x,train_y)


C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Keegan Fernandes\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[09:49:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
y_pred = model.predict(test_x)
predictions = [round(values) for values in y_pred]

In [ ]:
accuracy = accuracy_score(test_y , predictions)
precision = precision_score(test_y , predictions)
auc = roc_auc_score(test_y, y_pred)

In [ ]:
print("accuracy :" , accuracy)
print("precision :" , accuracy)
print("auc :" , auc)